In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash"

In [ ]:
gpt_client = OpenAI(
    api_key=openai_api_key
)

claude_client = OpenAI(
    api_key=anthropic_api_key,
    base_url="https://api.anthropic.com/v1/"
)

gemini_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
# Tests

messages = [{"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Howdy partner!"}]

gpt_response = gpt_client.chat.completions.create(
    model=gpt_model,
    messages=messages,
    temperature=0.5
)
print(f"GPT: {gpt_response.choices[0].message.content}")

claude_response = claude_client.chat.completions.create(
    model=claude_model,
    messages=messages,
    temperature=0.5
)
print(f"Claude: {claude_response.choices[0].message.content}")

gemini_response = gemini_client.chat.completions.create(
    model=gemini_model,
    messages=messages,
    temperature=0.5
)
print(f"Gemini: {gemini_response.choices[0].message.content}")

In [ ]:
gpt_system = "You are 'user_1'. You are snarky, entitled, and argumentative. Your role is to try and argue about anything and everything, and always have the last word, and never back down."
claude_system = "You are 'user_2'. You are a sharp debater. You always debate every argument, and you do everything you can to be the debate winner. You don't stop until you have the upper hand."
gemini_system = "You are 'user_3'. You are a mediator, coach and philosopher. Your job is to bring two sides to an agreement and have them stop arguing."

In [ ]:
apis = {
    "gpt": {
        "name": "gpt",
        "user_name": "Gapetto",
        "client": gpt_client,
        "model": gpt_model,
        "system": gpt_system,
        "messages": [],
    },
    "claude": {
        "name": "claude",
        "user_name": "Claudia",
        "client": claude_client,
        "model": claude_model,
        "system": claude_system,
        "messages": [],
    },
    "gemini": {
        "name": "gemini",
        "user_name": "Germione",
        "client": gemini_client,
        "model": gemini_model,
        "system": gemini_system,
        "messages": []
    }
}

In [ ]:
def call_llm(name: str):
    llm = apis[name]
    context = [{"role": "system", "content": llm["system"]}]
    
    gpt_role, gpt_name = ("assistant", "") if name == "gpt" else ("user", f'{apis["gpt"]["user_name"]}: ')
    claude_role, claude_name = ("assistant", "") if name == "claude" else ("user", f'{apis["claude"]["user_name"]}: ')
    gemini_role, gemini_name = ("assistant", "") if name == "gemini" else ("user", f'{apis["gemini"]["user_name"]}: ')
    
    for gpt, claude, gemini in zip(apis["gpt"]["messages"], apis["claude"]["messages"], apis["gemini"]["messages"]):
        context.append({"role": gpt_role, "content": f"{gpt_name}{gpt}"})
        context.append({"role": claude_role, "content": f"{claude_name}{claude}"})
        context.append({"role": gemini_role, "content": f"{gemini_name}{gemini}"})
    
    for i, key in enumerate(apis.keys()):
        if key != name:
            if len(apis[key]["messages"]) > len(llm["messages"]):
                context.append({"role": "user", "content": f'{apis[key]["user_name"]}: {apis[key]["messages"][-1]}'})
    
    response = llm["client"].chat.completions.create(
        model=llm["model"],
        messages=context,
        temperature=0.7
    )

    message = response.choices[0].message.content
    llm["messages"].append(message)
    return message

In [ ]:
apis["gpt"]["messages"] = ["Hi"]
apis["claude"]["messages"] = ["Hi"]
apis["gemini"]["messages"] = ["Lord of the Rings or Harry Potter?"] # Obviously LOTR.

In [ ]:
gpt_response = call_llm("gpt")
display(Markdown(f"### Gapetto:\n{gpt_response}\n\n"))

claude_response = call_llm("claude")
display(Markdown(f"### Claudia:\n{claude_response}\n\n"))

gemini_response = call_llm("gemini")
display(Markdown(f"### Germione:\n{gemini_response}\n\n"))

In [ ]:
apis["gpt"]["messages"] = ["Hi"]
apis["claude"]["messages"] = ["Hi"]
apis["gemini"]["messages"] = ["Lord of the Rings or Harry Potter?"]

for i in range(5):
    display(Markdown(f"## Round {i+1}:\n\n"))
    
    gpt_response = call_llm("gpt")
    display(Markdown(f"### Gapetto:\n{gpt_response}\n\n"))

    claude_response = call_llm("claude")
    display(Markdown(f"### Claudia:\n{claude_response}\n\n"))

    gemini_response = call_llm("gemini")
    display(Markdown(f"### Germione:\n{gemini_response}\n\n"))